In [129]:
import numpy as np
import pandas as pd
import os

## Load Data

In [151]:
# Data OHLCV
ohlcv = pd.read_csv('/home/dat/Desktop/RL project/env/main/mirae_OS_ts/1m/VN30F1M.csv',index_col = 0 ,parse_dates= True)
ohlcv = ohlcv.resample('5min').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum',
    'buy_vol':'sum',
    'sell_vol': 'sum',
    'fr_buy_vol': 'sum',
    'fr_sell_vol': 'sum',
    'val': 'sum'
}).dropna()
ohlcv.index =  ohlcv.index + pd.Timedelta(minutes=5)
ohlcv.head(10)

,Open,High,Low,Close,Volume,buy_vol,sell_vol,fr_buy_vol,fr_sell_vol,val
Date,,,,,,,,,,
2022-09-05 02:05:00,1294.0,1295.2,1293.5,1293.9,3624.0,842.0,2637.0,4.0,187.0,4690470.2
2022-09-05 02:10:00,1294.0,1294.4,1293.7,1293.7,1236.0,654.0,580.0,11.0,5.0,1599541.8
2022-09-05 02:15:00,1293.8,1293.8,1292.9,1293.2,1728.0,761.0,967.0,0.0,17.0,2234781.8
2022-09-05 02:20:00,1293.0,1294.7,1293.0,1294.7,2125.0,1237.0,835.0,21.0,4.0,2749823.7
2022-09-05 02:25:00,1294.7,1294.8,1293.8,1294.0,2225.0,724.0,1370.0,8.0,22.0,2879728.9
2022-09-05 02:30:00,1293.9,1294.3,1293.3,1293.4,1154.0,227.0,914.0,0.0,59.0,1492805.0
2022-09-05 02:35:00,1293.3,1293.7,1293.1,1293.6,1370.0,406.0,893.0,15.0,4.0,1772010.9
2022-09-05 02:40:00,1293.6,1294.3,1293.5,1293.9,1152.0,748.0,312.0,11.0,6.0,1490611.9
2022-09-05 02:45:00,1294.0,1294.0,1293.5,1293.9,662.0,340.0,322.0,3.0,4.0,856500.0


In [155]:
# Data alpha pnl
folder_path = '/home/dat/Desktop/RL project/env/main/pnl'
alpha_pnl = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        alpha_pnl[filename[:-4]] = pd.read_csv(file_path, index_col = 0, parse_dates=True)[['TOTAL','value']].reindex(ohlcv.index).ffill()
alpha_pnl

{'VN_event_mfi_gap_co_5m_1':                             TOTAL         value
 Date                                           
 2022-09-05 02:05:00  0.000000e+00  0.000000e+00
 2022-09-05 02:10:00 -6.725760e+06 -6.725760e+06
 2022-09-05 02:15:00 -7.642909e+06 -1.436867e+07
 2022-09-05 02:20:00  2.292873e+07  8.560058e+06
 2022-09-05 02:25:00 -1.070007e+07 -2.140015e+06
 ...                           ...           ...
 2023-12-29 07:20:00  2.209579e+07  2.195391e+10
 2023-12-29 07:25:00 -8.034833e+06  2.194587e+10
 2023-12-29 07:30:00 -1.205225e+07  2.193382e+10
 2023-12-29 07:35:00 -2.008708e+06  2.193181e+10
 2023-12-29 07:50:00 -1.275530e+07  2.191906e+10
 
 [17038 rows x 2 columns],
 'DMI_top30stock_10m_1':                      TOTAL         value
 Date                                    
 2022-09-05 02:05:00    NaN           NaN
 2022-09-05 02:10:00    0.0  0.000000e+00
 2022-09-05 02:15:00    0.0  0.000000e+00
 2022-09-05 02:20:00    0.0  0.000000e+00
 2022-09-05 02:25:00    0.0  0

In [156]:
folder_path = '/home/dat/Desktop/RL project/env/main/pos'
alpha_pos = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        temp = pd.read_csv(file_path, index_col = 0, parse_dates=True)[['VN30F1M']]
        temp.rename(columns = {'VN30F1M':'position'}, inplace = True)
        alpha_pos[filename[:-4]] = temp.reindex(ohlcv.index).ffill()

In [157]:
list(alpha_pnl.keys())

['VN_event_mfi_gap_co_5m_1',
 'DMI_top30stock_10m_1',
 'InverseFisher_RSI_alpha_5m_1',
 'VN_gap_co_volume_support_5m_1',
 'alpha_fr_ba_10m_1',
 'Buysellgap_top20stock_15m_1',
 'alpha_event_rsi_bgd10m_5m_1',
 'alpha_doublegap_aroon_15m_1',
 'sar_of_rsi_frbs_12m_1',
 'cmo_softmax_mean_std_ab_uw_5m_1',
 'alpha_ret_gapau_foreign_5m_1',
 'Frama_alpha_30mbigger_5m_1',
 'VN_buy_sell_vol_twap_5m_1',
 'wave_period_ba_bgd15m_5m_1',
 'Fisher_bidask_20m_1',
 'cci_heikin_bid_ask_resample15m_5m_1',
 'Rank_TSF_openclose_stock_30m_1',
 'alpha_hlc_delta_fr_10m_1',
 'alpha_typical_ba_ab_5m_1',
 'rsi_kari_heiken_abuw_20m_1',
 'pgo_indi_ab_uw_bgd10m_5m_1',
 'mfi_gap_abuw_resample30m_15m_1',
 'VN_buy_sell_combat_5m_1',
 'zscore_vwap_ab_uw_bgd15m_5m_1',
 'Ultimate_Oscillator_arbit_unwind_cutloss_5m_1',
 'alpha_gap_vwap_med_au_5m_1']

In [158]:
alpha_pos['Rank_TSF_openclose_stock_30m_1'].head(60)

,position
Date,
2022-09-05 02:05:00,NaN
2022-09-05 02:10:00,NaN
2022-09-05 02:15:00,NaN
2022-09-05 02:20:00,NaN
2022-09-05 02:25:00,NaN
2022-09-05 02:30:00,0.0000
2022-09-05 02:35:00,0.0000
2022-09-05 02:40:00,0.0000
2022-09-05 02:45:00,0.0000


In [159]:
18000*25

450000